In [0]:
# импорт библиотек
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

from google.colab import drive
from google.colab import drive
from sklearn.metrics import classification_report

In [0]:
# обращаемся гугл диску
drive.mount('/content/gdrive')

# !ls -la "/content/gdrive/My Drive/acoustic_event_analysis"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# путь к файлам
path = "/content/gdrive/My Drive/acoustic_event_analysis/"

In [0]:
# считываем дынные из файла
with open(path + 'all_data_mel.pickle', 'rb') as f:
  all_data_mel = pickle.load(f)

In [0]:
# приводим признаки к одной длине
# определяем максимальную длину
max_column = max([x.shape[1] for x in all_data_mel])
# max_line = max([x.shape[0] for x in all_data_mel])
for i in range(len(all_data_mel)):
  # создаем нулевой массив с недостающими нулями
  zero = np.zeros((all_data_mel[i].shape[0], max_column - all_data_mel[i].shape[1]))
  # конкатенируем массивы
  all_data_mel[i] = np.hstack((all_data_mel[i], zero))

In [0]:
# сохранаем all_data
with open(path + 'array_norm_data_mel.pickle', 'wb') as f_data:
    pickle.dump(norm_data_mel, f_data)

In [0]:
# подгружаем сохраненные признаки и метки
with open(path + 'array_norm_data_mel.pickle', 'rb') as f:
    X = pickle.load(f)
  
with open(path + 'all_label.pickle', 'rb') as f:
  y = pickle.load(f)

In [0]:
# разбиваем признаки и метки на test и train
# для этого необходимо создать словарь, где ключь - это метки, 
# значения - это список массивов признаков для каждой метки
all_dict = {}
for i in range(len(X)):
    if y[i] in all_dict:
        all_dict.update({y[i]: all_dict[y[i]] + [X[i]]})
    else:
        all_dict.update({y[i]: [X[i]]}) 

In [0]:
# делим данные на test(30%), train(70%)
train_data = []
train_label = []
test_data = []
test_label = []

for k_new_dict, v_new_dict in all_dict.items():
  count = 0
  for i in v_new_dict:
    count += 1
    if count <= round(len(v_new_dict)*0.7):
      train_data.append(i) # добавляем в словарь массив признаков
      train_label.append(k_new_dict) # добавляем в словарь метку массива признаков
    else:
      test_data.append(i) # добавляем в словарь массив признаков
      test_label.append(k_new_dict)  # добавляем в словарь метку массива признаков

In [0]:
print(len(train_data), len(train_label))
print(len(test_data), len(test_label))

3978 3978
1705 1705


In [0]:
# Приводим метки и признаки к нужному формату

# преобразуем метки из списков в массивы
y_train_new = np.asarray(train_label)
y_test_new = np.asarray(test_label)

# преобразуем признаки из списков в массивы
train_data_new = np.asarray(train_data, dtype=np.float32)
test_data_new = np.asarray(test_data, dtype=np.float32)

train_data_new = np.expand_dims(train_data_new, 1)
test_data_new = np.expand_dims(test_data_new, 1)

# формируем тензоры
inputs_train_new = torch.from_numpy(train_data_new)
inputs_test_new = torch.from_numpy(test_data_new)
labels_train = torch.from_numpy(y_train_new)
labels_test = torch.from_numpy(y_test_new)

print('Форма обучающих признаков: {}\nФорма тестовых признаков: {}\nФорма обучающих меток: {}\nФорма тестовых меток: {}\n '
      .format(inputs_train_new.shape, inputs_test_new.shape, labels_train.shape, labels_test.shape))

Форма обучающих признаков: torch.Size([3978, 1, 128, 469])
Форма тестовых признаков: torch.Size([1705, 1, 128, 469])
Форма обучающих меток: torch.Size([3978])
Форма тестовых меток: torch.Size([1705])
 


In [0]:
# задаем параметры для сети

num_epochs = 100
num_classes = 41
batch_size = 100
learning_rate = 0.001

In [0]:
# создаем архитектуру сети

class ConvNet(nn.Module):
    # вариант 1
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(32 * 117 * 32, 1000)
        self.fc2 = nn.Linear(1000, 41)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    ## вариант 2
    # def __init__(self):
    #     super(ConvNet, self).__init__()
    #     self.conv1 = nn.Conv2d(1, 6, 5)
    #     self.pool = nn.MaxPool2d(2, 2)
    #     self.conv2 = nn.Conv2d(6, 16, 5)
    #     self.drop_out = nn.Dropout()
    #     self.fc1 = nn.Linear(16 * 29 * 114, 100)
    #     # self.fc2 = nn.Linear(100, 50)
    #     self.fc3 = nn.Linear(100, 41)

    # def forward(self, x):
    #     x = self.pool(F.relu(self.conv1(x)))
    #     x = self.pool(F.relu(self.conv2(x)))
    #     x = x.reshape(x.size(0), -1)
    #     x = self.drop_out(x)
    #     x = F.relu(self.fc1(x))
    #     # x = F.relu(self.fc2(x))
    #     x = self.fc3(x)
    #     return x

model = ConvNet()

In [14]:
criterion = nn.CrossEntropyLoss() # уже встроена функция активации softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(inputs_train_new)
loss_list = []
acc_list = []
for epoch in range(num_epochs):

    current_step = 1
    finish = 0
    for i in range(0, len(inputs_train_new), batch_size):
      if len(inputs_train_new) - i <= batch_size:
        start = i
        finish = len(inputs_train_new)
      else:
        start = i
        finish += batch_size
      # Run the forward pass
      outputs = model(inputs_train_new[start:finish])
      loss = criterion(outputs, labels_train[start:finish])
      loss_list.append(loss.item())

      # Backprop and perform Adam optimisation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Track the accuracy
      total = len(labels_train[start:finish])
      _, predicted = torch.max(outputs.data, 1)
      correct = (predicted == labels_train[start:finish]).sum().item()
      acc_list.append(correct / total)

      if i % total_step == 0:
        print('Epoch [{}/{}], Step [{}-{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
              .format(epoch + 1, num_epochs, i, finish, total_step, loss.item(),
                      (correct / total) * 100))

Epoch [1/100], Step [0-100/3978], Loss: 3.6713, Accuracy: 4.00%
Epoch [2/100], Step [0-100/3978], Loss: 3.6689, Accuracy: 44.00%
Epoch [3/100], Step [0-100/3978], Loss: 3.6721, Accuracy: 44.00%
Epoch [4/100], Step [0-100/3978], Loss: 3.6783, Accuracy: 44.00%
Epoch [5/100], Step [0-100/3978], Loss: 3.6901, Accuracy: 1.00%
Epoch [6/100], Step [0-100/3978], Loss: 3.7195, Accuracy: 2.00%
Epoch [7/100], Step [0-100/3978], Loss: 3.7104, Accuracy: 1.00%
Epoch [8/100], Step [0-100/3978], Loss: 3.6963, Accuracy: 4.00%
Epoch [9/100], Step [0-100/3978], Loss: 3.6296, Accuracy: 4.00%
Epoch [10/100], Step [0-100/3978], Loss: 3.7030, Accuracy: 7.00%
Epoch [11/100], Step [0-100/3978], Loss: 3.6669, Accuracy: 5.00%
Epoch [12/100], Step [0-100/3978], Loss: 3.6482, Accuracy: 6.00%
Epoch [13/100], Step [0-100/3978], Loss: 3.6424, Accuracy: 5.00%
Epoch [14/100], Step [0-100/3978], Loss: 3.6722, Accuracy: 5.00%
Epoch [15/100], Step [0-100/3978], Loss: 3.5873, Accuracy: 6.00%
Epoch [16/100], Step [0-100/397

In [15]:
# Тестирум модель
correct = 0
total = len(labels_test)
y_pred_all = []
with torch.no_grad():
    finish = 0
    for i in range(0, total, batch_size):
      # print(i)
      if total - i <= batch_size:
        start = i
        finish = total
      else:
        start = i
        finish += batch_size
      outputs = model(inputs_test_new[start:finish])
      _, predicted = torch.max(outputs, 1)
      y_pred_all.append([x for x in predicted.data.numpy()])
      correct += (predicted == labels_test[start:finish]).sum().item()

print('Accuracy of the network on the {} test images: {:.2f}%'.format(total, 100*(correct/total)))

Accuracy of the network on the 1705 test images: 22.93%


In [16]:
# преобразуем список предсказаний
y_pred_list = [y for x in y_pred_all for y in x]
len(y_pred_list)

1705

In [17]:
# выводим метрики
print(classification_report(test_label, y_pred_list, digits = 4))

              precision    recall  f1-score   support

           0     0.5000    0.2083    0.2941        24
           1     0.1111    0.0784    0.0920        51
           2     0.3000    0.1250    0.1765        24
           3     0.1250    0.0769    0.0952        26
           4     0.4286    0.2885    0.3448        52
           5     0.1273    0.1250    0.1261        56
           6     0.0000    0.0000    0.0000        20
           7     0.2857    0.3774    0.3252        53
           8     0.1364    0.0577    0.0811        52
           9     0.1111    0.0816    0.0941        49
          10     0.4035    0.4182    0.4107        55
          11     0.1429    0.1091    0.1237        55
          12     0.1667    0.1957    0.1800        46
          13     0.7222    0.4815    0.5778        54
          14     0.5926    0.4571    0.5161        35
          15     0.2857    0.1176    0.1667        17
          16     0.4286    0.6316    0.5106        57
          17     0.1034    

Для сравнения подгрузим ранее полученные матрики при использовании
класификатора SVM

In [0]:
prediction_test = np.load(path + 'prediction_test_v1.npy')

In [0]:
# считываем метки из файла
with open(path + 'label_test.pickle', 'rb') as f:
  test_label = pickle.load(f)

In [23]:
print(classification_report(test_label, prediction_test, digits = 4))

              precision    recall  f1-score   support

           0     0.6842    0.5417    0.6047        24
           1     0.1250    0.0392    0.0597        51
           2     0.1176    0.2500    0.1600        24
           3     0.0500    0.0385    0.0435        26
           4     0.6000    0.4615    0.5217        52
           5     0.2609    0.1071    0.1519        56
           6     0.1154    0.1500    0.1304        20
           7     0.1385    0.3396    0.1967        53
           8     0.0938    0.0577    0.0714        52
           9     0.1765    0.0612    0.0909        49
          10     0.2721    0.6727    0.3874        55
          11     0.2344    0.2727    0.2521        55
          12     0.1216    0.1957    0.1500        46
          13     0.6333    0.7037    0.6667        54
          14     0.9048    0.5429    0.6786        35
          15     0.7500    0.3529    0.4800        17
          16     0.3770    0.4035    0.3898        57
          17     0.3333    

Как можно заметить классификация методом SVM показывает 
f_scora_weighted_avg = 23,5 % что на 1,27% больше
чем CNN.